In [307]:
import numpy as np
import cv2 as cv

In [308]:
def cv_show(name,img):
    cv.imshow(name, img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [309]:
# 正确答案
ANSWER_KEY = {0: 1,1: 2, 2: 0, 3: 2, 4: 3}

In [310]:
#读入图片并查看
image = cv.imread('image.png')
cv_show('image', image)

In [311]:
#高斯滤波
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
blurred = cv.GaussianBlur(gray, (5, 5), 0)
cv_show('blurred', blurred)

In [312]:
#边缘检测
edged = cv.Canny(blurred, 75, 200)
cv_show('edged', edged)

In [313]:
#轮廓检测，检测出外轮廓
cnts = cv.findContours(edged.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)[1]
contours_img = image.copy()
cv.drawContours(contours_img, cnts, -1, [0, 0, 255], 3)
cv_show('contours_img',contours_img)
print(np.array(cnts).shape)

(21,)


C:\py\lib\site-packages\ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [314]:
#通过比较轮廓周长、面积的方式筛选出最外围的轮廓
cnts = sorted(cnts, key=cv.contourArea, reverse=True)

In [315]:
#进行透视变换只需要4个点（左上，右上，右下，左下）的位置即可完成，故需要找到此外轮廓的近似多边形的坐标。
peri = cv.arcLength(cnts[0], True)
approx = cv.approxPolyDP(cnts[0], 0.02 * peri, True)
print(approx)

[[[131 206]]

 [[119 617]]

 [[448 614]]

 [[430 208]]]


In [316]:
#将得到的点坐标按照左上，右上，右下，左下排序
#为了使现在得到的点与投射完成后的点的位置一一对应，在这里要先将这些点按一定顺序排列。
pts = approx.reshape(4, 2)
rect = np.zeros((4, 2), dtype=np.float32)
s = np.sum(pts, axis=1)
rect[0] = pts[np.argmin(s)]
rect[2] = pts[np.argmax(s)]
diff = np.diff(pts, axis=1)
rect[1] = pts[np.argmin(diff)]
rect[3] = pts[np.argmax(diff)]

In [317]:
#获取变换后对应坐标位置
# 变换后，图像的长和宽应该变为：
# 长 = max（变换前左边长，变换前右边长）
# 宽 = max（变换前上边长，变换前下边长）
# 设变换后图像的左上角位置为原点位置。
# 获取坐标点
tl, tr, br, bl = rect
# 计算输入的w和h值
widthA = np.sqrt(((br[0]-bl[0]) ** 2) + ((br[1]-bl[1]) ** 2))
widthB = np.sqrt(((tr[0]-tl[0]) ** 2) + ((tr[1]-tl[1]) ** 2))
maxWidth = max(int(widthA), int(widthB))

heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
maxHeight = max(int(heightA), int(heightB))
# 变换后对应坐标位置
dst = np.array([[0, 0],
               [maxWidth-1, 0],
               [maxWidth-1, maxHeight-1],
               [0, maxHeight-1]], dtype=np.float32)

In [318]:
#计算变换矩阵并做透视变换
H = cv.getPerspectiveTransform(rect, dst)
warped = cv.warpPerspective(gray, H, (maxWidth, maxHeight))
cv_show('warped',warped)

In [319]:
#二值化
thresh = cv.threshold(warped, 0, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)[1]
cv_show('thresh',thresh)

In [320]:
#找到轮廓并将它们涂黑
thresh_Contours = thresh.copy()
cnts = cv.findContours(thresh_Contours, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)[1]
print(len(cnts))
cv.drawContours(thresh_Contours, cnts, -1, (0, 0, 0), 3)
cv_show('thresh_Contours',thresh_Contours)

82


In [321]:
# 找到包含选项的25个圆形轮廓
# 先找出所有轮廓的近似矩形。
# 计算矩形的长和宽，按照一定标准进行筛选。
questionCnts = []
# 遍历
for c in cnts:
    # 计算比例和大小
    (x, y, w, h) = cv.boundingRect(c)
    ar = w / float(h)

    # 根据实际情况制定标准
    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

In [322]:
# 对25个圆形轮廓按照从上到下进行排序，划分出5组（即代表五道题）
boundingBoxes = [cv.boundingRect(c) for c in questionCnts]
(questionCnts, boundingBoxes) = zip(*sorted(zip(questionCnts, boundingBoxes), key=lambda b: b[1][1], reverse=False))
#在这里使用的相关函数（zip、lambda等）可以参考lambda函数的用法、Python内置函数 – zip(), sorted(), filter()和map()、Python中星号变量的特殊用法。

In [323]:
# 对每组中的5个选项所代表的轮廓进行排序。
# 制作一个mask来遍历每一个选项。
# 制作一个全黑mask。
# 用25个圆形轮廓对mask进行填充（一次只用一个轮廓）。
# 用填充完的mask对图像进行与操作，遍历后会得到25个只包含一个选项的图像。
# 每5组图像进行比较，由于被选的选项的白色区域更多，所以通过计算非零点数量来判断考生选择的答案。
# 与正确答案进行对比，得出分数。
# 每排有5个选项
correct = 0
print(len(questionCnts))
warpeds=cv.cvtColor(warped.copy(),cv.COLOR_GRAY2BGR)
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    # 排序
    boundingBoxes = [cv.boundingRect(c) for c in questionCnts[i:i + 5]]
    (cnts, boundingBoxes) = zip(*sorted(zip(questionCnts[i:i + 5], boundingBoxes),
                                        key=lambda b: b[1][0], reverse=False))
    bubbled = None

    # 遍历每一个结果
    for (j, c) in enumerate(cnts):
        # 使用mask来判断结果
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv.drawContours(mask, [c], -1, 255, -1) #-1表示填充
#         cv_show('mask',mask)
        # 通过计算非零点数量来算是否选择这个答案
        mask = cv.bitwise_and(thresh, thresh, mask=mask)
#         cv_show('mask', mask)
        total = cv.countNonZero(mask)

        # 通过阈值判断
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)

    # 对比正确答案
    color = (0, 0, 255)
    k = ANSWER_KEY[q]

    # 判断正确
    if k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1
#     print(k)
    cv.drawContours(warpeds, [cnts[k]], -1, color, 3)
# 绘图
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv.putText(warpeds, "{:.2f}%".format(score), (10, 30),
    cv.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)
cv.imshow("Original", image)
cv.imshow("Exam", warpeds)
cv.waitKey(0)
cv.destroyAllWindows()    

25
[INFO] score: 60.00%
